In [1]:
%load_ext autoreload 
#allows you to reload this module without having to restart kernel
%aimport rugpeek_functions 
import matplotlib.pyplot as plt
import numpy as np
import lmfit as lf
import matplotlib as mpl
import sys

Define some data filenames to read in

In [2]:
data_file_ferric_soret_shg0 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_SHG_10kHz_TA_1_matrix"
data_file_ferric_soret_shg1 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_SHG_10kHz_TA_2_matrix"
data_file_ferric_soret_shg2 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_SHG_10kHz_TA_3_matrix"

data_file_ferric_soret_fun0 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_FUN_10kHz_TA_1_matrix"
data_file_ferric_soret_fun1 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_FUN_10kHz_TA_2_matrix"
data_file_ferric_soret_fun2 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_FUN_10kHz_TA_3_matrix"

solvent_ferric_soret_shg = "../data/aarhus_TA_jan24/2024-01-16/milliQ_409nm_SHG_chirp_1_matrix"
solvent_ferric_soret_fun = "../data/aarhus_TA_jan24/2024-01-16/milliQ_409nm_FUN_chirp_1_matrix"

Load them as instances of the Rug class - here wrapped within an average function in case you want to average more than one (commented out in example)

In [6]:
%autoreload 1
rp = rugpeek_functions
#JDP create an instance of the Rug class
ferric_soret_shg = rp.RugTools.average_rugs([rp.Rug(data_file_ferric_soret_shg0, ".dat")])
                                            # rp.Rug(data_file_ferric_soret_shg1, ".dat"),
                                            # rp.Rug(data_file_ferric_soret_shg2, ".dat")])

ferric_soret_fun = rp.RugTools.average_rugs([rp.Rug(data_file_ferric_soret_fun0, ".dat")])
                                            # rp.Rug(data_file_ferric_soret_fun1, ".dat"),
                                            # rp.Rug(data_file_ferric_soret_fun2, ".dat")])

solvent_soret_shg = rp.Rug(solvent_ferric_soret_shg, ".dat")
solvent_soret_fun = rp.Rug(solvent_ferric_soret_fun, ".dat")


Apply dispersion corrections. Get them from the solvent only file and then apply the coefficients to the actual data.

In [7]:
%matplotlib qt5

solvent_soret_shg.get_dispersion_correction_eye(degree=2)
coefs_soret_shg = solvent_soret_shg.dispersion_coefs
solvent_soret_shg.apply_dispersion_correction(coefs=coefs_soret_shg)
ferric_soret_shg.apply_dispersion_correction(coefs=coefs_soret_shg)
ferric_soret_shg.offset(coef=coefs_soret_shg[-1])


solvent_soret_fun.get_dispersion_correction_eye(degree=2)
coefs_soret_fun = solvent_soret_fun.dispersion_coefs
ferric_soret_fun.apply_dispersion_correction(coefs=coefs_soret_fun)
solvent_soret_fun.apply_dispersion_correction(coefs=coefs_soret_fun)
ferric_soret_fun.offset(coef=coefs_soret_fun[-1])

plt.close('all')


Offset corrected
Offset corrected


Deep copy the corrected rugs so that we can easily get back to them if we break something later. Then stitch together the SHG and FUN regions into one big dataset.

In [9]:
import copy
ferric_soret_shg_mod = copy.deepcopy(ferric_soret_shg)
ferric_soret_fun_mod = copy.deepcopy(ferric_soret_fun)
solvent_soret_shg_mod = copy.deepcopy(solvent_soret_shg)
solvent_soret_fun_mod = copy.deepcopy(solvent_soret_fun)

ferric_soret_rugs = [ferric_soret_shg_mod, ferric_soret_fun_mod]
ferric_soret_total = rp.RugTools.combine_rugs_wavelengths(ferric_soret_rugs, "ferric_soret_combined_matrix")

solvent_soret_rugs = [solvent_soret_shg_mod, solvent_soret_fun_mod]
solvent_soret_total = rp.RugTools.combine_rugs_wavelengths(solvent_soret_rugs, "solvent_soret_combined_matrix")


Cut out wavelengths that are contaminated by scattering/pump light etc. Best to background subtract after you've done this.

In [13]:
%autoreload 1
ferric_soret_total.cut_wavelengths([(395,430), (512,518)], fill=0)

ferric_soret_total.background_subtract()
ferric_soret_total.peek()

#JDP uncomment if you want to undo operations on the rug.
#ferric_soret_total.reset_matrix()

(239, 453) (239, 453)


Compute and plot an SVD - see functions for parameters.

In [30]:
%autoreload 1
plt.close('all')

#JDP uncomment if you want to limit stuff
#ferric_soret_total.limit_times(tmin=10, tmax=1000)
#ferric_soret_total.limit_wavelengths(wlmin=413, wlmax=700)
                                                       
ferric_soret_total.compute_SVD(threshold=100)

ferric_soret_total.plot_SVD()


Taking threshold for singular value significance as 100x larger than smallest singular value (37.139299489).
Found 2 significant singular values
90.36 % of data variance described by component 1.
4.58 % of data variance described by component 2.
Overall, 94.94 % of data variance described by the first 2 components.
Leaving 5.06 % of data variance described by the the remaining 237 components which are below threshold (roughly 0.02 % per component).


Small helper functions that let you get traces and spectra at given times/wavelengths - example below.

In [44]:

wavelengths = [460, 455, 450, 445, 440, 435, 390, 385, 380, 375, 370]
decays = ferric_soret_total.get_time_trace(wavelengths, plot=True)

times = [-1, 0, 1, 10, 100]
spectra = ferric_soret_total.get_wavelength_trace(times, plot=True)



In [18]:
#### Fitting the SVD components

%autoreload 2
t = ferric_soret_total.times

#plt.close('all')
def normalise(x):
    a = (x-np.min(x))/np.max(x)
    return a

def normalise_scale(x):
    return x/np.max(x)

def inverse_uncertainty(x, s):
    s_inv = s / (x**2)
    return s_inv
    
traces = ferric_soret_total.relevant_kinetics[0:3]

ncpts = 2

IRF_center_val = 0
IRF_center_min = -0.3
IRF_center_max = 0.3
IRF_center= [IRF_center_val, IRF_center_min, IRF_center_max]

IRF_sigma_val = 0.4
IRF_sigma_min = 0
IRF_sigma_max = 0.5
IRF_sigma= [IRF_sigma_val, IRF_sigma_min, IRF_sigma_max]

decay_amplitude_center = [1, -1, 1]
decay_amplitude_min = [-5, -5, -5]
decay_amplitude_max = [5, 5, 5]

decay_gamma_center = [1, 1, 1]
decay_gamma_min = [1E-6, 1E-6, 1E-6]
decay_gamma_max = [20, 20, 20]
mods = []

for idx, trace in enumerate(traces):
    y = normalise_scale(trace)
    print(f'Fitting component {idx+1} of the SVD.')
    print(f'Fitting to {ncpts} exponential decays.')
    
    for n in range(ncpts):
        if n == 0:
            mod = lf.models.ExponentialGaussianModel(prefix='p'+str(n)+'_')
            pars = mod.make_params(center = dict(value=IRF_center[0], min=IRF_center[1], 
                                                 max=IRF_center[2], vary=True),
                        sigma = dict(value=IRF_sigma[0], min=IRF_sigma[1], 
                                     max=IRF_sigma[2], vary=True),
                        amplitude = dict(value=decay_amplitude_center[n],min=decay_amplitude_min[n],
                                         max=decay_amplitude_max[n], vary=True), 
                        gamma = dict(value=decay_gamma_center[n], min=decay_gamma_min[n], 
                                     max=decay_gamma_max[n], vary=True))
            model = mod
        else:
            mod = lf.models.ExponentialGaussianModel(prefix='p'+str(n)+'_')
            pars.update(mod.make_params(center = dict(expr='p0_center', vary=False),
                        sigma = dict(expr='p0_sigma', vary=False),
                        amplitude = dict(value=decay_amplitude_center[n],min=decay_amplitude_min[n],
                                         max=decay_amplitude_max[n], vary=True), 
                        gamma = dict(value=decay_gamma_center[n], min=decay_gamma_min[n], 
                                     max=decay_gamma_max[n])))
            model = model + mod


    out = model.fit(y, pars, x=t, method='least_squares')
    
    plt.figure()
    ax = out.plot_fit()
    ax.set_xlabel('Time (ps)')
    ax.set_ylabel(r'$\Delta OD')
    ax.set_xscale('linear')
    t0 = out.params['p'+str(0)+'_center'].value
    FWHM = out.params['p'+str(0)+'_sigma'].value
    print(f'Fitted IRF as centered at {t0} ps and with {FWHM} ps width.')
    for n in range(ncpts):
        l1 = np.round(1/out.params['p'+str(n)+'_gamma'].value, 4) 
        l1_err = np.round(inverse_uncertainty(l1, out.params['p'+str(n)+'_gamma'].stderr), 4)
        a1 = np.round(out.params['p'+str(n)+'_amplitude'].value,4)
        print(f'Lifetime is: {l1} +/- {l1_err} ps. Corresponding amplitude is {a1}.')



Fitting component 1 of the SVD.
Fitting to 2 exponential decays.
Fitted IRF as centered at 0.01667708522414615 ps and with 0.08500084165781328 ps width.
Lifetime is: 3.9432 +/- 0.0002 ps. Corresponding amplitude is 2.2301.
Lifetime is: 0.6142 +/- 0.1242 ps. Corresponding amplitude is 0.3559.
Fitting component 2 of the SVD.
Fitting to 2 exponential decays.
Fitted IRF as centered at 0.04473723878728863 ps and with 0.09181105901512855 ps width.
Lifetime is: 0.3463 +/- 0.5835 ps. Corresponding amplitude is 0.7363.
Lifetime is: 5.7126 +/- 0.0002 ps. Corresponding amplitude is -2.5391.


In [60]:
#  paramdata needs to include all the parameters that are used to fit one dataset
# if using inbuilt lmfit models the names need to match what is expected 
# numbers are just used to differentiate them, and are replaced with prefixes later 


# need to fix this so it checks the length of the params dictionary
paramdata = {'center': [0, -0.3, 0.3, True],
             'sigma': [0.4, 0.0, 0.5, True],
             'gamma': [1, 1E-6, 20, True],
             'amplitude': [0, -5, 5, True],

             'center2': [0, -0.3, 0.3, True],
             'sigma2': [0.4, 0.0, 0.5, True],
             'gamma2': [1, 1E-6, 20, True],
             'amplitude2': [0, -5, 5, True],

             'center3': [0, -0.3, 0.3, True],
             'sigma3': [0.4, 0.0, 0.5, True],
             'gamma3': [1, 1E-6, 30, True],
             'amplitude3': [0, -5, 5, True]}
           #  'c': [0, 0, 2, False]
           # }


#need a way to check that the number of suppplied params matches expected components

#these are bits that are common to each component and to each dataset
fixed_vars = ['center','sigma']

#these are bits that apply to one whole dataset, but vary across different datasets ('offsets')
offset_vars = [] 

#these are bits that vary across components, but are commmon to each dataset
fixed_data_vars = ['gamma'] 

# wavelengths of time slices to fit to
wavelengths = [455, 450, 445, 440, 435, 390, 385, 380]
traces = np.array(ferric_soret_total.get_time_trace(wavelengths))
t = ferric_soret_total.times
ncpts = 1
vars_per_cpt = 4

#call the global fitter to fit to a sum of exponential gaussians
out = rp.RugFits.global_fitter(t, traces, paramdata, None, None, ncpts=ncpts, vars_per_cpt=vars_per_cpt, 
                    prefixdata='d', prefixcpt='c', model=lf.models.ExponentialGaussianModel,
                    method='leastsq', fixed_vars=fixed_vars, offset_vars=offset_vars, fixed_data_vars=fixed_data_vars)

#plot the report. make print true for details
rp.RugFits.plot_fit_result(out, t, traces, wavelengths, ncpts, print_report=False)

Parameter dictionary passed in is longer than expected. Ignoring everything after the 4th element.
Lifetime 0 is: 2.4712 +/- 0.0009 ps.


In [55]:
len(paramdata)

12

In [99]:
for i, _ in enumerate(traces):
    resid = out.residual.reshape(len(traces), len(traces[0]))
    fitdata = traces[i] - resid[i]
    plt.figure()
    plt.plot(t, fitdata, color='C0')
    plt.scatter(t, traces[i], color='C1')
    plt.xlim(-1,30)
    if i == 0:
        for n in range(ncpts):
            l1 = np.round(1/out.params['d'+str(i)+'c'+str(n)+'_gamma'].value, 4) 
            l1_err = 0#np.round(inverse_uncertainty(l1, out.params['d'+str(i)+'c'+str(n)+'_gamma'].stderr), 4)
            print(f'Lifetime {n} is: {l1} +/- {l1_err} ps.')

print(lf.fit_report(out))

Lifetime 0 is: 3.5652 +/- 0 ps.
Lifetime 1 is: 0.6741 +/- 0 ps.
Lifetime 2 is: 0.0336 +/- 0 ps.
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 60000
    # data points      = 1912
    # variables        = 29
    chi-square         = 1.38019873
    reduced chi-square = 7.3298e-04
    Akaike info crit   = -13772.7916
    Bayesian info crit = -13611.6703
##  Warning: uncertainties could not be estimated:
    d5c0_amplitude:  at boundary
[[Variables]]
    d0c0_center:     0.02470574 (init = 0)
    d0c0_sigma:      0.08825032 (init = 0.4)
    d0c0_gamma:      0.28048623 (init = 1)
    d0c0_amplitude:  1.13954752 (init = 0)
    d0c1_center:     0.02470574 == 'd0c0_center'
    d0c1_sigma:      0.08825032 == 'd0c0_sigma'
    d0c1_gamma:      1.48337907 (init = 1)
    d0c1_amplitude:  0.32631057 (init = 0)
    d0c2_center:     0.02470574 == 'd0c0_center'
    d0c2_sigma:      0.08825032 == 'd0c0_sigma'
    d0c2_gamma:      29.7397161 (init = 1)
    d0c2_amplitude:  0

In [25]:


DAS1 = 2.1752*ferric_soret_total.singular_values[0]*ferric_soret_total.relevant_spectra[0] \
        -3.108*ferric_soret_total.singular_values[1]*ferric_soret_total.relevant_spectra[1]

DAS2 = 0.4311*ferric_soret_total.singular_values[0]*ferric_soret_total.relevant_spectra[0] \
     + 1.0255*ferric_soret_total.singular_values[1]*ferric_soret_total.relevant_spectra[1]

plt.figure()
plt.plot(ferric_soret_total.wavelengths, DAS1, label='DAS, 4.5ps cpt')
plt.plot(ferric_soret_total.wavelengths, DAS2, label='DAS, 0.5ps cpt')
plt.legend()

In [ ]:
ferric_soret_total_cut = copy.deepcopy(ferric_soret_total)

#ferric_soret_total_cut.cut_wavelengths(400,415)
ferric_soret_total_cut.cut_wavelengths(510, 520)


In [99]:
ferric_soret_total.peek()

In [ ]:
times = [-0.1,0.1, 0.5, 1., 5, 5, 50]
traces = []
plt.close('all')
fig = plt.figure()
ax = fig.gca()
colours = np.flipud(plt.cm.viridis(np.linspace(0, 1, len(times))))

insax = ax.inset_axes([0.38, 0.5, 0.4, 0.4])

for idx, time in enumerate(times):
    ax.plot(ferric_soret_total_cut.wavelengths, 
             ferric_soret_total_cut.get_wavelength_trace(time), label=str(time)+'ps', color=colours[idx])

    insax.plot(ferric_soret_total_cut.wavelengths, 
             ferric_soret_total_cut.get_wavelength_trace(time), label=str(time)+'ps', color=colours[idx])


insax.set_xlim(520, 640)
insax.set_ylim(-0.5,1.5)
insax.set_yticks([0, 1])
insax.set_xlabel('Wavelength (nm)')
insax.set_ylabel(r'$\Delta OD$')
insax.set_xticks([520, 580, 640])
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'$\Delta OD$')
ax.set_xlim(350,700)
ax.set_ylim(-3,12)
ax.legend()



In [ ]:
ferric_soret_total.compute_SVD()

In [ ]:
%autoreload 1
plt.close('all')
plt.figure()
sing = ferric_soret_total.singular_values
for idx, spectrum in enumerate(ferric_soret_total.principal_spectra[0:3]):
    rp.RugTools.cut_wavelengths_spectrum(ferric_soret_total.wavelengths, spectrum, 510, 520)
    #rp.RugTools.cut_wavelengths_spectrum(ferric_soret_total.wavelengths, spectrum, 405, 415)
    plt.plot(ferric_soret_total.wavelengths, spectrum, label='Component '+str(idx+1))

plt.legend()
plt.xlabel('Wavelength (nm)')
plt.ylabel(r'$\Delta OD$')
plt.xlim(350, 700)

In [ ]:
plt.figure()
plt.plot(ferric_soret_total.times, ferric_soret_total.get_time_trace(410))
plt.plot(solvent_soret_shg.times, solvent_soret_shg.get_time_trace(410))


In [ ]:
ferric_soret_total.peek(min_max=(-10, 1))